# Q2 
### Data fetching & Preprocessing

In [2]:
import numpy as np
import os
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
import joblib
from natsort import natsorted
import string
import regex as re
import os
from pathlib import Path

# give the path to dir of dataset.
path =Path("./Humor,Hist,Media,Food")
path

PosixPath('Humor,Hist,Media,Food')

In [7]:

import joblib
#initialize the variables neede

stemmer = PorterStemmer()
fileno = 0
positionalIndex ={}
fileNumberMap={}
    
#read the files from the folder in alphabetical order.
file_names = natsorted(os.listdir(path))
for file in file_names:
  #open the file in utf8 format
    f = open(str(path)+"/"+file,'r', encoding ="utf8", errors ="surrogateescape")
    #spli the lines for any multiple new lines
    data = f.read().split('\n\n')
    #initialize the position of each word in the file
    pos=1
    """for every line in the file 
       make the string to lowercase,
       remove any numbers from the line 
       make a tokenizer to convert the string to tokens, here used the regular expression tokenizer that removes the special characters
       removed the stop words from the tokens."""
    for line in data: 
      line = line.lower()
      line= re.sub(r'\d+','',line)
      tokenizer = RegexpTokenizer(r"\w+")
      tokenList = tokenizer.tokenize(line)
      tokensWithoutStopWords = [word for word in tokenList if not word in stopwords.words('english')]
      """ for each word in tokens, stem the words(remove the trailing characters) 
      """
      for word in tokensWithoutStopWords:
        word= stemmer.stem(word)
        if word in positionalIndex:
          if fileno in positionalIndex[word]:
            positionalIndex[word][fileno].append(pos)
          else:
            positionalIndex[word][fileno] = [pos]
        else:
          positionalIndex[word]={fileno:[pos]}
          # positionalIndex[word]
        pos+=1
    fileNumberMap[fileno] = file


    fileno+=1


In [ ]:
# fileNumberMap
joblib.dump(positionalIndex,"positionalIndex.joblib")
joblib.dump(fileNumberMap,"FileNumberMap.joblib")


In [53]:
# def phraseQuery(line):
#     posIndex = joblib.load("positionalIndex.joblib")
#     fileNumberMap = joblib.load("FileNumberMap.joblib")

#     # for word in phrase: preprocess the query.
#     line = line.lower()
#     line= re.sub(r'\d+','',line)
#     line =re.sub(r'[^\w\s]','',line)
#     tokenizer = RegexpTokenizer(r"\w+")
#     tokenList = tokenizer.tokenize(line)
#     tokensWithoutStopWords = [word for word in tokenList if not word in stopwords.words('english')]

#     print("Words To Find------->\t",tokensWithoutStopWords)
    
#     fileNames = []

#     posList =[]
#     indexForWord = {}
#     """for every word in the tokens, iterate over the positional Index and get the file number and positions for a word.
#         Make a smaller dictionary containing only the files and position for the given words.
#     """
#     for pos,word in enumerate(tokensWithoutStopWords):
#         for docId,positions in posIndex[PorterStemmer().stem(word)].items():

#             if docId not in indexForWord:
#                 indexForWord[docId]=positions
#             else:
#                 for i in positions:
#                     indexForWord[docId].append(i)
    
#     todel =[]
#     """For the file numbers and positions in the dictionary, remove the file number that contains positions less than the number of words in the query. 
#         make a list that contains files with the same and pop them from the dictionary."""
#     for k,v in indexForWord.items():
#         if(len(indexForWord[k])<len(tokensWithoutStopWords)):
#             todel.append(k)

#     for i in todel:
#         indexForWord.pop(i)

#     """Iterate over the file numbers and the positions in dictionary, sort the positions and check for consecutive positions.
#         If the number of consecutive postions is not empty then the file contains the query."""
#     for docId,positions in indexForWord.items():
#         positions.sort()
#         # print(positions)
#         consecutiveList = []
#         for i in range(len(positions)-1):
#             if(positions[i+1]-positions[i] ==1):
#                 consecutiveList.append(positions[i+1]-positions[i])

#         if(len(consecutiveList)):
#             fileNames.append([docId,fileNumberMap[docId]])
    
#         # print(consecutiveList)
#     print("Number Of Docs Retreived------>\t",len(fileNames),"\nFound  DocID & Name ------>\t",fileNames)


# phraseQuery("These artists are \"cultural jammers\"")


# phraseQuery("laughing at your husband because ")



In [55]:
def phraseQuery(line):
    posIndex = joblib.load("positionalIndex.joblib")
    fileNumberMap = joblib.load("FileNumberMap.joblib")

    # for word in phrase: preprocess the query.
    line = line.lower()
    line= re.sub(r'\d+','',line)
    line =re.sub(r'[^\w\s]','',line)
    tokenizer = RegexpTokenizer(r"\w+")
    tokenList = tokenizer.tokenize(line)
    tokensWithoutStopWords = [word for word in tokenList if not word in stopwords.words('english')]

    print("Words To Find------->\t",tokensWithoutStopWords)
    
    fileNames = []

    posList =[]
    indexForWord = {}
    """If we only have one word to find"""
    if(len(tokensWithoutStopWords)==1): 
        for docId,positions in posIndex[PorterStemmer().stem(tokensWithoutStopWords[0])].items():
            fileNames.append([docId,fileNumberMap[docId]])
        print("Number Of Docs Retreived------>\t",len(fileNames),"\nFound  DocID & Name ------>\t",fileNames)
        return



    """for every word in the tokens, iterate over the positional Index and get the file number and positions for a word.
        Make a smaller dictionary containing only the files and position for the given words.
    """
    for pos,word in enumerate(tokensWithoutStopWords):
        for docId,positions in posIndex[PorterStemmer().stem(word)].items():

            if docId not in indexForWord:
                indexForWord[docId]=positions
            else:
                for i in positions:
                    indexForWord[docId].append(i)
    
    todel =[]
    """For the file numbers and positions in the dictionary, remove the file number that contains positions less than the number of words in the query. 
        make a list that contains files with the same and pop them from the dictionary."""
    for k,v in indexForWord.items():
        if(len(indexForWord[k])<len(tokensWithoutStopWords)):
            todel.append(k)

    for i in todel:
        indexForWord.pop(i)

    """Iterate over the file numbers and the positions in dictionary, sort the positions and check for consecutive positions.
        If the number of consecutive postions is not empty then the file contains the query."""
    for docId,positions in indexForWord.items():
        positions.sort()
        consecutiveList = []
        for i in range(len(positions)-1):
            # if(positions[i+1]-positions[i] ==1):
                consecutiveList.append(positions[i+1]-positions[i])

        # print(positions)
        """Find the maximum number of consecutive 1s and if that is equal to length f tokens -1, add file"""
        mx=0
        for i in range(len(consecutiveList)):
            c=0
            while(i<len(consecutiveList) and consecutiveList[i]==1):
                i+=1
                c+=1
            mx = max(c,mx)
        if(mx== len(tokensWithoutStopWords)-1):
            fileNames.append([docId,fileNumberMap[docId]])
    
        # print(consecutiveList)
    print("Number Of Docs Retreived------>\t",len(fileNames),"\nFound  DocID & Name ------>\t",fileNames)


phraseQuery("These artists are \"cultural jammers\"")


phraseQuery("laughing at your husband because ")

phraseQuery("maiden")




Words To Find------->	 ['artists', 'cultural', 'jammers']
Number Of Docs Retreived------>	 1 
Found  DocID & Name ------>	 [[650, 'merry.txt']]
Words To Find------->	 ['laughing', 'husband']
Number Of Docs Retreived------>	 2 
Found  DocID & Name ------>	 [[418, 'gd_gal.txt'], [659, 'misery.hum']]
Words To Find------->	 ['maiden']
Number Of Docs Retreived------>	 12 
Found  DocID & Name ------>	 [[129, 'bnbeg2.4.txt'], [245, 'comic_st.gui'], [295, 'devils.jok'], [318, 'dromes.txt'], [345, 'epi_bnb.txt'], [547, 'jokes'], [549, 'jokes.txt'], [593, 'limerick.jok'], [889, 'reeves.txt'], [978, 'strine.txt'], [1035, 'top10st1.txt'], [1119, 'x-drinks.txt']]
